In [50]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split #from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import numpy as np

In [2]:
import os
train=pd.read_csv('C:\\Users\\jalan\\Desktop\\CapStone Retrieved\\TrainTestMethod\\Traincorpwithoutsidecorpus.csv')
test=pd.read_csv('C:\\Users\\jalan\\Desktop\\CapStone Retrieved\\TrainTestMethod\\Test.csv')

In [3]:
X_train = train.stemmedjoin
y_train = train.labels
X_test = test.stemmedjoin
y_test = test.labels
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((9318,), (9318,), (2057,), (2057,))

In [4]:
y_train.value_counts()

neutral     6582
positive    1728
negative    1008
Name: labels, dtype: int64

In [5]:
vect = TfidfVectorizer(stop_words=None, max_features=200000, ngram_range=(1, 2))
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)

In [6]:
tfidf_pd=pd.DataFrame(X_train_dtm.todense(),columns=vect.get_feature_names())
tfidf_pd.shape

(9318, 55549)

In [7]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<2057x55549 sparse matrix of type '<class 'numpy.float64'>'
	with 21404 stored elements in Compressed Sparse Row format>

In [8]:
#over sampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=777)
X_ROS, y_ROS = ros.fit_sample(X_train_dtm, y_train)
#under sampling
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=777)
X_RUS, y_RUS = rus.fit_sample(X_train_dtm, y_train)

Naive Bayes

In [9]:
NB = MultinomialNB()
%time NB.fit(X_train_dtm, y_train)
y_pred_class_NB = NB.predict(X_test_dtm)
# calculate precision and recall
print(classification_report(y_test, y_pred_class_NB))

Wall time: 116 ms
             precision    recall  f1-score   support

   negative       0.00      0.00      0.00       111
    neutral       0.80      1.00      0.89      1649
   positive       1.00      0.01      0.01       297

avg / total       0.79      0.80      0.72      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
%time NB.fit(X_ROS, y_ROS)
y_pred_class_NBROS = NB.predict(X_test_dtm)
# calculate precision and recall
print(classification_report(y_test, y_pred_class_NBROS))

Wall time: 67.8 ms
             precision    recall  f1-score   support

   negative       0.18      0.32      0.23       111
    neutral       0.88      0.71      0.79      1649
   positive       0.32      0.59      0.42       297

avg / total       0.76      0.67      0.70      2057



In [11]:
%time NB.fit(X_RUS, y_RUS)
y_pred_class_NBRUS = NB.predict(X_test_dtm)
# calculate precision and recall
print(classification_report(y_test, y_pred_class_NBRUS))

Wall time: 12.9 ms
             precision    recall  f1-score   support

   negative       0.18      0.41      0.25       111
    neutral       0.89      0.59      0.71      1649
   positive       0.26      0.62      0.37       297

avg / total       0.76      0.59      0.64      2057



Logistic Regression

In [12]:
lr = LogisticRegression(multi_class='multinomial',solver='lbfgs',C=15)#15 seems to be around optimal value. Regularization parameter

In [13]:
%time lr.fit(X_train_dtm, y_train)
# make class predictions for X_test_dtm - Test
y_pred_class_lr = lr.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_lr))

Wall time: 3.85 s
             precision    recall  f1-score   support

   negative       0.27      0.14      0.18       111
    neutral       0.84      0.92      0.88      1649
   positive       0.47      0.31      0.37       297

avg / total       0.76      0.79      0.77      2057



In [14]:
%time lr.fit(X_ROS, y_ROS)
# make class predictions for X_test_dtm - Test
y_pred_class_lrROS = lr.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_lrROS))

Wall time: 3.81 s
             precision    recall  f1-score   support

   negative       0.24      0.21      0.22       111
    neutral       0.87      0.86      0.87      1649
   positive       0.43      0.46      0.45       297

avg / total       0.77      0.77      0.77      2057



In [15]:
%time lr.fit(X_RUS, y_RUS)
# make class predictions for X_test_dtm - Test
y_pred_class_lrRUS = lr.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_lrRUS))

Wall time: 3.67 s
             precision    recall  f1-score   support

   negative       0.16      0.44      0.24       111
    neutral       0.89      0.63      0.74      1649
   positive       0.29      0.60      0.39       297

avg / total       0.77      0.61      0.66      2057



SGD Classifier

In [16]:
sgd_clf = SGDClassifier(random_state=42)
%time sgd_clf.fit(X_train_dtm, y_train)
# make class predictions for X_test_dtm - Test
y_pred_class_sgd = sgd_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_sgd))

C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Wall time: 186 ms
             precision    recall  f1-score   support

   negative       0.26      0.06      0.10       111
    neutral       0.83      0.95      0.89      1649
   positive       0.47      0.21      0.29       297

avg / total       0.75      0.80      0.76      2057



In [17]:
sgd_clf.fit(X_ROS, y_ROS)
y_pred_class_sgdROS = sgd_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_sgdROS))

             precision    recall  f1-score   support

   negative       0.28      0.29      0.28       111
    neutral       0.88      0.83      0.85      1649
   positive       0.39      0.51      0.44       297

avg / total       0.77      0.75      0.76      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [18]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_RUS, y_RUS)
y_pred_class_sgdRUS = sgd_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_sgdRUS))

             precision    recall  f1-score   support

   negative       0.15      0.47      0.23       111
    neutral       0.88      0.61      0.72      1649
   positive       0.28      0.55      0.37       297

avg / total       0.76      0.59      0.64      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


DecisionTreeClassifier

In [138]:
tree_clf = DecisionTreeClassifier(min_samples_split=50,min_samples_leaf=100)
%time tree_clf.fit(X_train_dtm, y_train)
# make class predictions for X_test_dtm - Test
y_pred_class_tree = tree_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_tree))

Wall time: 788 ms
             precision    recall  f1-score   support

   negative       0.20      0.01      0.02       111
    neutral       0.82      0.97      0.89      1649
   positive       0.48      0.15      0.23       297

avg / total       0.74      0.80      0.75      2057



In [135]:
tree_clf.fit(X_ROS, y_ROS)
y_pred_class_treeROS = tree_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_treeROS))

             precision    recall  f1-score   support

   negative       0.10      0.28      0.14       111
    neutral       0.87      0.61      0.72      1649
   positive       0.25      0.48      0.33       297

avg / total       0.74      0.58      0.63      2057



In [139]:
#tree_clf = DecisionTreeClassifier(max_depth=175,min_samples_split=100)
tree_clf.fit(X_RUS, y_RUS)
y_pred_class_treeRUS = tree_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_treeRUS))

             precision    recall  f1-score   support

   negative       0.06      0.40      0.11       111
    neutral       0.83      0.44      0.57      1649
   positive       0.24      0.37      0.29       297

avg / total       0.70      0.43      0.51      2057



Bagging Classifier

In [183]:
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train_dtm, y_train)
y_pred_bagg = bag_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_bagg))

             precision    recall  f1-score   support

   negative       0.00      0.00      0.00       111
    neutral       0.80      1.00      0.89      1649
   positive       0.00      0.00      0.00       297

avg / total       0.64      0.80      0.71      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [185]:
bag_clf.fit(X_ROS, y_ROS)
y_pred_baggROS = bag_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_baggROS))

             precision    recall  f1-score   support

   negative       0.07      0.39      0.12       111
    neutral       0.83      0.62      0.71      1649
   positive       0.38      0.27      0.32       297

avg / total       0.72      0.56      0.62      2057



In [322]:
bag_clf.fit(X_RUS, y_RUS)
y_pred_baggRUS = bag_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_baggRUS))

             precision    recall  f1-score   support

   negative       0.07      0.39      0.12       111
    neutral       0.82      0.62      0.70      1649
   positive       0.35      0.25      0.29       297

avg / total       0.71      0.55      0.61      2057



Random Forest Classifier

In [194]:
RF = RandomForestClassifier(n_estimators=500,  n_jobs=-1)
%time RF.fit(X_train_dtm, y_train)
# make class predictions for X_test_dtm - Test
y_pred_class_RFtree = RF.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_RFtree))

Wall time: 2min 49s
             precision    recall  f1-score   support

   negative       0.25      0.04      0.06       111
    neutral       0.82      0.97      0.89      1649
   positive       0.49      0.13      0.20       297

avg / total       0.74      0.80      0.74      2057



In [196]:
RF.fit(X_ROS, y_ROS)
y_pred_class_RFtreeROS = RF.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_RFtreeROS))

             precision    recall  f1-score   support

   negative       0.24      0.06      0.10       111
    neutral       0.82      0.95      0.88      1649
   positive       0.42      0.19      0.26       297

avg / total       0.73      0.79      0.75      2057



In [195]:
RF.fit(X_RUS, y_RUS)
y_pred_class_RFtreeRUS = RF.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_RFtreeRUS))

             precision    recall  f1-score   support

   negative       0.14      0.25      0.18       111
    neutral       0.86      0.73      0.79      1649
   positive       0.29      0.43      0.35       297

avg / total       0.74      0.67      0.69      2057



ExtraTreesClassifier (Extremely Random Random forest)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ETC = ExtraTreesClassifier(n_estimators=1000,  n_jobs=-1)
%time ETC.fit(X_train_dtm, y_train)
y_pred_class_ETCtree = ETC.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_ETCtree))

In [ ]:
ETCros.fit(X_ROS, y_ROS)
y_pred_class_ETCtreeROS = ETCros.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_ETCtreeROS))

In [198]:
ETCrus.fit(X_RUS, y_RUS)
y_pred_class_ETCtreeRUS = ETCrus.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_ETCtreeRUS))

             precision    recall  f1-score   support

   negative       0.20      0.37      0.26       111
    neutral       0.87      0.77      0.81      1649
   positive       0.33      0.45      0.38       297

avg / total       0.75      0.70      0.72      2057



Support Vector Classifiers

In [91]:
from sklearn import svm
#svm = svm.SVC(gamma=1)
svm = svm.SVC(kernel="poly", degree=1,gamma=1, coef0=1)
%time svm.fit(X_train_dtm, y_train)
# make class predictions for X_test_dtm - Test
y_pred_class_svm = svm.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_svm))

Wall time: 13.5 s
             precision    recall  f1-score   support

   negative       0.23      0.05      0.09       111
    neutral       0.83      0.96      0.89      1649
   positive       0.52      0.20      0.29       297

avg / total       0.75      0.80      0.76      2057



In [88]:
svm.fit(X_ROS, y_ROS)
y_pred_class_svmROS = svm.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_svmROS))

             precision    recall  f1-score   support

   negative       0.21      0.14      0.17       111
    neutral       0.86      0.87      0.87      1649
   positive       0.43      0.46      0.45       297

avg / total       0.77      0.77      0.77      2057



In [92]:
svm.fit(X_RUS, y_RUS)
y_pred_class_svmRUS = svm.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_svmRUS))

             precision    recall  f1-score   support

   negative       0.17      0.41      0.24       111
    neutral       0.88      0.67      0.76      1649
   positive       0.30      0.55      0.39       297

avg / total       0.76      0.64      0.68      2057



AdaBoost

In [319]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=5000,algorithm="SAMME.R", learning_rate=0.01)
ada_clf.fit(X_RUS, y_RUS)
y_pred_class_adaRUS = ada_clf.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_adaRUS))

             precision    recall  f1-score   support

   negative       0.13      0.22      0.16       111
    neutral       0.85      0.77      0.81      1649
   positive       0.30      0.38      0.34       297

avg / total       0.73      0.68      0.70      2057



All Results

In [323]:
#Results=pd.DataFrame(np.zeros((15,3)),columns=['negative','neutral','positive']) # Creting empty dataframe 
#Results['Technique']=0
negative=(classification_report(y_test, y_pred_class_NBRUS)[82:86],classification_report(y_test, y_pred_class_lrRUS)[82:86],\
 classification_report(y_test, y_pred_class_sgdRUS)[82:86], classification_report(y_test, y_pred_class_treeRUS)[82:86],\
 classification_report(y_test, y_pred_baggRUS)[82:86], classification_report(y_test, y_pred_class_RFtreeRUS)[82:86],\
 classification_report(y_test, y_pred_class_ETCtreeRUS)[82:86], classification_report(y_test, y_pred_class_svmRUS)[82:86],\
 classification_report(y_test, y_pred_class_adaRUS)[82:86])

In [324]:
neutral=(classification_report(y_test, y_pred_class_NBRUS)[135:139],classification_report(y_test, y_pred_class_lrRUS)[135:139],\
 classification_report(y_test, y_pred_class_sgdRUS)[135:139], classification_report(y_test, y_pred_class_treeRUS)[135:139],\
 classification_report(y_test, y_pred_baggRUS)[135:139], classification_report(y_test, y_pred_class_RFtreeRUS)[135:139],\
 classification_report(y_test, y_pred_class_ETCtreeRUS)[135:139], classification_report(y_test, y_pred_class_svmRUS)[135:139],\
 classification_report(y_test, y_pred_class_adaRUS)[135:139])

In [350]:
positive=(classification_report(y_test, y_pred_class_NBRUS)[188:192],classification_report(y_test, y_pred_class_lrRUS)[188:192],\
 classification_report(y_test, y_pred_class_sgdRUS)[188:192], classification_report(y_test, y_pred_class_treeRUS)[188:192],\
 classification_report(y_test, y_pred_baggRUS)[188:192], classification_report(y_test, y_pred_class_RFtreeRUS)[188:192],\
 classification_report(y_test, y_pred_class_ETCtreeRUS)[188:192], classification_report(y_test, y_pred_class_svmRUS)[188:192],\
 classification_report(y_test, y_pred_class_adaRUS)[188:192])

In [351]:
Results = pd.DataFrame([negative,neutral,positive],columns=['Negative Binomial','Log Reg','Stochastic Gradient','Decision Tree', 'Bagging', 'Random Forest','ExtraTreesClassifier',\
          'Support Vector Classifiers', 'Ada Boost'])
Results.index=['negative','neutral','positive',]
Results

,Negative Binomial,Log Reg,Stochastic Gradient,Decision Tree,Bagging,Random Forest,ExtraTreesClassifier,Support Vector Classifiers,Ada Boost
negative,0.41,0.44,0.47,0.40,0.39,0.25,0.37,0.41,0.22
neutral,0.59,0.63,0.61,0.44,0.62,0.73,0.77,0.67,0.77
positive,0.62,0.60,0.55,0.37,0.25,0.43,0.45,0.55,0.38


Ensemble Model

In [142]:
from sklearn.ensemble import VotingClassifier
eAlg = VotingClassifier(estimators=[('NB',NB),('LR',lr),('SGD',sgd_clf),('DT',tree_clf),('svc',svm)],
                        voting='hard')#,weights=[2,2,1,2,1])
eAlg.fit(X_ROS, y_ROS)
y_pred_class_vsROS = eAlg.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_vsROS))

C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


             precision    recall  f1-score   support

   negative       0.24      0.22      0.23       111
    neutral       0.87      0.85      0.86      1649
   positive       0.42      0.48      0.45       297

avg / total       0.77      0.76      0.77      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [150]:
eAlg = VotingClassifier(estimators=[('LR',lr),('NB',NB),('SGD',sgd_clf),('DT',tree_clf),('svc',svm)],
                        voting='hard',weights=[1,1,1,1,1])
eAlg.fit(X_RUS, y_RUS)
y_pred_class_vsRUS = eAlg.predict(X_test_dtm)
print(classification_report(y_test, y_pred_class_vsRUS))

C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


             precision    recall  f1-score   support

   negative       0.15      0.47      0.22       111
    neutral       0.88      0.63      0.73      1649
   positive       0.31      0.56      0.40       297

avg / total       0.76      0.61      0.66      2057



C:\Users\jalan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
#fpr, tpr, thresholds = roc_curve(y_test, y_pred_class_lrRUS)

In [32]:
#test['predit_lr']=y_pred_class_lr
#test['predit_lrROS']=y_pred_class_lrROS
#test['predit_lrRUS']=y_pred_class_lrRUS

In [33]:
#test.head()

In [34]:
#os.getcwd()
#test.to_csv("test1.csv")